<span style="float:left;">Licence CC BY-NC-ND</span><span style="float:right;">Thierry Parmentelat &amp; Arnaud Legout&nbsp;<img src="media/inria-25-alpha.png" style="display:inline"></span><br/>

# Boucles d'événements

[`asyncio.get_event_loop().run_complete(<future>)`](https://docs.python.org/3/library/asyncio-eventloop.html#asyncio.AbstractEventLoop.run_until_complete)

```
asyncio.get_event_loop().run_complete(
    asyncio.gather(coro1, coro2, ...)
))
```

## Ajout de traitements

`asyncio.ensure_future(coro)`

* pour ajouter une coroutine dans la boucle
* avant ou après le lancement de la boucle

[`asyncio.get_event_loop().run_forever()`](https://docs.python.org/3/library/asyncio-eventloop.html#asyncio.AbstractEventLoop.run_forever)

# Utilitaire #1

In [ ]:
import asyncio

from asynchelpers import start_timer, show_timer

In [ ]:
import time

start_timer()
time.sleep(0.5)
show_timer('un message')

# Utilitaire #2

In [ ]:
from asynchelpers import sequence

start_timer()
asyncio.get_event_loop().run_until_complete(
    sequence("hello", 1)
)


# Boucle sans fin

In [ ]:
import asyncio

loop = asyncio.get_event_loop()
asyncio.ensure_future(sequence('foo', delay=0.5))
asyncio.ensure_future(sequence('bar', delay=1))

In [ ]:
# on lance la boucle sans fin
# interrompre aver Kernel -> Interrupt 
# raccourci clavier : 'i' (doubler si nécessaire)
try:
    start_timer()
    asyncio.get_event_loop().run_forever()
except KeyboardInterrupt as e:
    print("bye")

# `run_until_complete` vs `run_forever`

* `run_until_complete`

   * prend exactement un argument
   * retourne la valeur


* `run_forever`
  * ne prend pas d'argument
  * ne retourne pas
  * orienté traitement massivement asynchrone

# `get_event_loop()`

* `get_event_loop()` 
  * boucle par défaut du thread courant
  * ne *crée pas* de boucle en dehors du thread principal

# `{set,new}_event_loop()`

In [ ]:
# pour repartir d'une boucle vierge:

# loop = asyncio.new_event_loop()
# asyncio.set_event_loop(loop)

# ou encore tout simplement
asyncio.set_event_loop(asyncio.new_event_loop())

## Ajout de traitements à la volée (*fork*)

![fork](w8-s5-av-fig1.png)

In [ ]:
# c1 et c3 sont très simples
async def c1():
    await sequence("c1", delay=2)

async def c3():
    await sequence("c3", delay=2)    

![fork](w8-s5-av-fig1.png)

In [ ]:
async def c2():
    show_timer('>>> c2')
    await asyncio.sleep(1)
    show_timer('=== ensure_future')
    asyncio.ensure_future(c3())
    await asyncio.sleep(1)
    show_timer('<<< c2')

# *fork* avec `ensure_future`

In [ ]:
# interrompre après 3s
asyncio.ensure_future(c1())
asyncio.ensure_future(c2())

start_timer()
try:
   asyncio.get_event_loop().run_forever()
except:
    print('interrupted')

![fork](w8-s5-av-fig1.png)

# `run_{until_complete,forever}`

In [ ]:
asyncio.set_event_loop(
    asyncio.new_event_loop())

In [ ]:
async def c1():
    await sequence('c1', delay=1)
async def c2():
    await sequence('c2', delay=2)
async def c3():
    await sequence('c3', delay=3)

In [ ]:
loop = asyncio.get_event_loop()

asyncio.ensure_future(c1())
asyncio.ensure_future(c3())

start_timer()
loop.run_until_complete(c2())
print("done 1")

In [ ]:
# interrompre après 1s
try:
    loop.run_forever()
except:
    print("interrupted")

# Résumé

* `get_event_loop()` 
  * accéde à la boucle courante
  * `new_event_loop()` et `set_event_loop()`

* `ensure_future()`

* `run_until_complete()` et `run_forever()`